### 필수과제1
- medium, source에 대한 예산 집행한다고 할 때, 크게 medium, source 예산 가정해서 만듬 (예산 금액 임의 선정, 10억 미만 선)
- 해당 예산 스프레드해서 비중, 가중치에 따라 예산 컬럼 만들고, 컬럼 기준으로 효율성 분석 (수치 계산)
- 예산, pageviews, hits, visits 에 따른 효율성 측정하는 지표 제작
- plotly 통해 해당 효율성 지표 시각화 보여주기
    - line, sunburst 등 가장 효율적으로 잘 보여주는 시각화 사용
    - 어떤 traffic 가장 효율적인지, 비효율적인지 확인
    - 두 개를 계산해서 시각화

In [1]:
import pandas as pd
import os

os.chdir(r'..')

In [63]:
import json

df = pd.read_csv('ga.csv') 

In [64]:
df['totals'] = df['totals'].apply(json.loads)
df['pageviews'] = df['totals'].apply(lambda x:int(x.get('pageviews'),0))
df['hits'] = df['totals'].apply(lambda x:int(x.get('hits'),0))
df['visits'] = df['totals'].apply(lambda x:int(x.get('visits'),0))

In [65]:
df['trafficSource'] = df['trafficSource'].apply(json.loads)
df['campaign'] = df['trafficSource'].apply(lambda x: x.get('campaign','(not set)'))
df['source'] = df['trafficSource'].apply(lambda x: x.get('source','(not set)'))
df['medium'] = df['trafficSource'].apply(lambda x: x.get('medium','(not set)'))

In [66]:
df.head(3)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime,pageviews,hits,visits,campaign,source,medium
0,Direct,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6.194190e+18,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",6194193421514403509_1472843572,Not Socially Engaged,"{'visits': '1', 'hits': '11', 'pageviews': '11...","{'campaign': '(not set)', 'source': '(direct)'...",1472843572,1,1472843572,11,11,1,(not set),(direct),(none)
1,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",5.327170e+18,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",5327166854580374902_1472844906,Not Socially Engaged,"{'visits': '1', 'hits': '11', 'pageviews': '10...","{'campaign': '(not set)', 'source': 'google', ...",1472844906,3,1472844906,10,11,1,(not set),google,organic
2,Referral,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8.885050e+18,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",8885051388942907862_1472827393,Not Socially Engaged,"{'visits': '1', 'hits': '13', 'pageviews': '11...","{'referralPath': '/', 'campaign': '(not set)',...",1472827393,7,1472827393,11,13,1,(not set),mall.googleplex.com,referral


In [67]:
agg_df_sm = df.groupby(['date','source','medium'])[['pageviews','hits','visits']].sum().reset_index()
agg_df_sm

,date,source,medium,pageviews,hits,visits
0,20160801,(direct),(none),341,441,8
1,20160801,basecamp.com,referral,49,69,1
2,20160801,gatewaycdi.com,referral,14,18,1
3,20160801,google,organic,216,372,7
4,20160801,mall.googleplex.com,referral,428,568,16
...,...,...,...,...,...,...
1753,20170801,google,cpc,20,28,1
1754,20170801,google,cpm,5,5,1
1755,20170801,google,organic,244,319,6
1756,20170801,mail.google.com,referral,16,16,1


In [68]:
val_date = '20160801'
val_source = 'basecamp.com'
val_medium = 'referral'

# 기존 데이터 확인 

df['date'] = df['date'].astype('str')
df['source'] = df['source'].str.strip()
df['medium'] = df['medium'].str.strip()

org_sum = df[(df['date']==val_date) & (df['source']==val_source) & (df['medium']==val_medium)][['pageviews','hits','visits']].sum()
print(f'org_sum : \n{org_sum} \n')

# 집계 데이터 확인

agg_df_sm['date'] = agg_df_sm['date'].astype('str')
agg_df_sm['source'] = agg_df_sm['source'].str.strip()
agg_df_sm['medium'] = agg_df_sm['medium'].str.strip()

agg_sum = agg_df_sm[(agg_df_sm['date']==val_date) & (agg_df_sm['source']==val_source) & (agg_df_sm['medium']==val_medium)][['pageviews','hits','visits']].sum()
print(f'agg_sum : \n{org_sum}')

org_sum : 
pageviews    49
hits         69
visits        1
dtype: int64 

agg_sum : 
pageviews    49
hits         69
visits        1
dtype: int64


In [69]:
agg_df_sm

,date,source,medium,pageviews,hits,visits
0,20160801,(direct),(none),341,441,8
1,20160801,basecamp.com,referral,49,69,1
2,20160801,gatewaycdi.com,referral,14,18,1
3,20160801,google,organic,216,372,7
4,20160801,mall.googleplex.com,referral,428,568,16
...,...,...,...,...,...,...
1753,20170801,google,cpc,20,28,1
1754,20170801,google,cpm,5,5,1
1755,20170801,google,organic,244,319,6
1756,20170801,mail.google.com,referral,16,16,1


### 예산 및 가중치 설정
광고 예산을 100,000,000 원으로 가정함
- (w1) Pageviews : 콘텐츠 조회수는 브랜드 인지도 확산과 관련 있으므로 가중치를 2순위로 설정 
- (w2) Hits : 서버 요청 수는 한 번의 방문에 따라 높게 측정될 수 있어 가중치를 3순위로 설정 
- (w3) Visits :  방문수는 실질적인 잠재 고객과 관련 있으므로 가중치를 1순위로 설정 

In [70]:
agg_df_sm.describe() # pageview : hits: visits = 185 : 237 : 6

,pageviews,hits,visits
count,1758.000000,1758.000000,1758.000000
mean,185.565984,237.598407,6.550057
std,202.979903,263.923322,6.782795
min,4.000000,4.000000,1.000000
25%,36.000000,45.000000,1.000000
50%,118.000000,149.000000,4.000000
75%,278.000000,356.000000,10.000000
max,1972.000000,2504.000000,42.000000


In [71]:
total_budget = 100000000
w1 = 2
w2 = 1
w3 = 70

In [72]:
agg_df_sm['total_w'] = (agg_df_sm['pageviews']*w1 + agg_df_sm['hits']*w2 + agg_df_sm['visits']*w3)
total_wegiht = agg_df_sm['total_w'].sum()
agg_df_sm['budget'] = (agg_df_sm['total_w']/total_wegiht)*total_budget

In [74]:
agg_df_sm

,date,source,medium,pageviews,hits,visits,total_w,budget
0,20160801,(direct),(none),341,441,8,1683,89702.685964
1,20160801,basecamp.com,referral,49,69,1,237,12631.929039
2,20160801,gatewaycdi.com,referral,14,18,1,116,6182.716323
3,20160801,google,organic,216,372,7,1294,68969.266570
4,20160801,mall.googleplex.com,referral,428,568,16,2544,135593.364879
...,...,...,...,...,...,...,...,...
1753,20170801,google,cpc,20,28,1,138,7355.300453
1754,20170801,google,cpm,5,5,1,85,4530.438685
1755,20170801,google,organic,244,319,6,1227,65398.214901
1756,20170801,mail.google.com,referral,16,16,1,118,6289.314880


### 효과성 지표
각 효과는 지표 1단위를 높이는데 사용된 예산으로 계산하여 지표 달성에 대한 비용으로 인식

In [108]:
agg_df_sm['pageviews_efficiency'] = agg_df_sm['budget']/agg_df_sm['pageviews']
agg_df_sm['hits_efficiency'] = agg_df_sm['budget']/agg_df_sm['hits']
agg_df_sm['visits_efficiency'] = agg_df_sm['budget']/agg_df_sm['visits']

In [109]:
agg_df_sm

,date,source,medium,pageviews,hits,visits,total_w,budget,pageviews_efficiency,hits_efficiency,visits_efficiency
0,20160801,(direct),(none),341,441,8,1683,89702.685964,263.057730,203.407451,11212.835745
1,20160801,basecamp.com,referral,49,69,1,237,12631.929039,257.794470,183.071435,12631.929039
2,20160801,gatewaycdi.com,referral,14,18,1,116,6182.716323,441.622595,343.484240,6182.716323
3,20160801,google,organic,216,372,7,1294,68969.266570,319.302160,185.401254,9852.752367
4,20160801,mall.googleplex.com,referral,428,568,16,2544,135593.364879,316.806927,238.720713,8474.585305
...,...,...,...,...,...,...,...,...,...,...,...
1753,20170801,google,cpc,20,28,1,138,7355.300453,367.765023,262.689302,7355.300453
1754,20170801,google,cpm,5,5,1,85,4530.438685,906.087737,906.087737,4530.438685
1755,20170801,google,organic,244,319,6,1227,65398.214901,268.025471,205.010078,10899.702483
1756,20170801,mail.google.com,referral,16,16,1,118,6289.314880,393.082180,393.082180,6289.314880


### 시각화
해당 지표는 값이 작을수록 효율적임을 의미한다는 것을 유의하여 해석해야 함 

In [111]:
melted_agg_df = agg_df_sm.melt(id_vars=['date','source','medium'],
               value_vars=['pageviews_efficiency','hits_efficiency','visits_efficiency'],
               var_name = 'metric',
               value_name = 'value')
melted_agg_df

,date,source,medium,metric,value
0,20160801,(direct),(none),pageviews_efficiency,263.057730
1,20160801,basecamp.com,referral,pageviews_efficiency,257.794470
2,20160801,gatewaycdi.com,referral,pageviews_efficiency,441.622595
3,20160801,google,organic,pageviews_efficiency,319.302160
4,20160801,mall.googleplex.com,referral,pageviews_efficiency,316.806927
...,...,...,...,...,...
5269,20170801,google,cpc,visits_efficiency,7355.300453
5270,20170801,google,cpm,visits_efficiency,4530.438685
5271,20170801,google,organic,visits_efficiency,10899.702483
5272,20170801,mail.google.com,referral,visits_efficiency,6289.314880


In [112]:
import plotly.express as px

fig = px.line(melted_agg_df,
        x='date',
        y='value',
        color = 'source',
        line_dash = 'medium',
        facet_row = 'metric',
        labels = {'value':'Metric Value', 'date':'Date'})
fig.update_layout(
    height = 900,
    legend_title_text = 'Source',
    xaxis = dict(tickangle=45))
fig.show()

In [124]:
fig = px.sunburst(
    agg_df_sm,
    path = ['medium','source'],
    values = 'pageviews_efficiency',
    color = 'pageviews_efficiency',
    color_continuous_scale = 'Blues')
fig.update_layout(
    height = 800,
    legend_title_text ='Source',
    xaxis = dict(tickangle=45))
fig.show()

# Pageview 측면에 있어, basecamp 는 비용효율이 가장 높고, phandroid는 비용효율이 가장 낮음

In [125]:
fig = px.sunburst(
    agg_df_sm,
    path = ['medium','source'],
    values = 'hits_efficiency',
    color = 'hits_efficiency',
    color_continuous_scale = 'Blues')
fig.update_layout(
    height = 800,
    legend_title_text ='Source',
    xaxis = dict(tickangle=45))
fig.show()

# Hits 측면에 있어, basecamp 는 비용효율이 가장 높고, phandroid는 비용효율이 가장 낮음 

In [126]:
fig = px.sunburst(
    agg_df_sm,
    path = ['medium','source'],
    values = 'visits_efficiency',
    color = 'visits_efficiency',
    color_continuous_scale = 'Blues')
fig.update_layout(
    height = 800,
    legend_title_text ='Source',
    xaxis = dict(tickangle=45))
fig.show()

# Visits 측면에 있어, seroundtable은 비용효율이 가장 높고, facebook은 비용효율이 가장 낮음 